In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from scipy.special import rel_entr
from scipy.stats import entropy
import scipy.stats
import math
import warnings 
warnings.filterwarnings("ignore") 

from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Wed Jun 29 01:33:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 32%   41C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   

# MongoDB

In [4]:
from pymongo import MongoClient
import pymongo

In [5]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_cn_2016 = db.rm_patent_cn_2016
    db_rm_us_2016 = db.rm_patent_us_2016
    db_rm_de_2016 = db.rm_patent_de_2016

    
except errors.ConnectionFailure as err:
    print(err)

# 2016

## Company數量

In [6]:
data_assignee = db_rm_de_2016.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [7]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [8]:
data_assignee = db_rm_cn_2016.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [9]:
data_assignee = db_rm_us_2016.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}},
    {"$limit":30}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"],data["count"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])

In [10]:
# final_df = pd.concat([df_family_de,df_family_cn,df_family_us],axis=0)

In [11]:
# 取總數前15家公司
df_family_cn.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(15)

,company,count
28,ZTE Corp,117
10,Huawei Technologies Co Ltd,111
0,Anhui Sharetronic Iot Technology Co Ltd,85
26,Xian Irain IoT Technology Service Co Ltd,84
20,Shanghai Langbo Communication Technology Co Ltd,82
14,Nanjing IoT Sensor Technology Co Ltd,54
17,SUZHOU XUANHE IOT TECHNOLOGY Co Ltd,53
13,Jiangsu IoT Research and Development Center,48
6,China Mobile Communications Group Co Ltd,38
18,Samsung Electronics Co Ltd,37


In [12]:
com_df = df_family_cn.groupby("company").sum().reset_index().sort_values("count",ascending=False).head(15)
com_df.reset_index(drop=True, inplace=True)

In [13]:
com_df["count_log"] = com_df["count"].apply(lambda x: np.log(x))
com_df

,company,count,count_log
0,ZTE Corp,117,4.762174
1,Huawei Technologies Co Ltd,111,4.709530
2,Anhui Sharetronic Iot Technology Co Ltd,85,4.442651
3,Xian Irain IoT Technology Service Co Ltd,84,4.430817
4,Shanghai Langbo Communication Technology Co Ltd,82,4.406719
5,Nanjing IoT Sensor Technology Co Ltd,54,3.988984
6,SUZHOU XUANHE IOT TECHNOLOGY Co Ltd,53,3.970292
7,Jiangsu IoT Research and Development Center,48,3.871201
8,China Mobile Communications Group Co Ltd,38,3.637586
9,Samsung Electronics Co Ltd,37,3.610918


公司名稱正規化

In [14]:
com_df["company"] = com_df.company.apply(lambda x: x.replace(" Inc","")\
                                                    .replace(" Co Ltd","")\
                                                    .replace(" Co. Ltd.","")\
                                                    .replace(" Co., Ltd.","")\
                                                    .replace(" Ltd","")\
                                                    .replace(" Corp","")\
                                                    .replace("..","")\
                                                    .replace("International Business Machines","IBM")\
                                                    .replace("Nippon Telegraph and Telephone","Nippon Telegraph & Tel")\
                                                    .replace("Alibaba Group Holding Ltd","Alibaba")\
                                                    .replace("ZTE Intelligent IoT Technology","ZTE")\
                                                    .replace("AT&T Intellectual Property I LP","AT&T")\
                                                    .replace("Microsoft Technology Licensing LLC","Microsoft")\
                                                    .replace("Telefonaktiebolaget LM Ericsson AB","Ericsson")\
                                                    .replace("Cisco Technology","Cisco Systems")\
                                                    .replace("Verizon Patent and Licensing","Verizon Communications")\
                                                    .replace("Tencent Technology Shenzhen","Tencent Holdings")\
                                                    .replace("China Mobile Communications Group","China Mobile")\
                                                    .replace("Alipay Hangzhou Information Technology","Alibaba Group")\
                                                    .replace("Beijing Xiaomi Mobile Software","Xiaomi"))

In [15]:
com_df

,company,count,count_log
0,ZTE,117,4.762174
1,Huawei Technologies,111,4.709530
2,Anhui Sharetronic Iot Technology,85,4.442651
3,Xian Irain IoT Technology Service,84,4.430817
4,Shanghai Langbo Communication Technology,82,4.406719
5,Nanjing IoT Sensor Technology,54,3.988984
6,SUZHOU XUANHE IOT TECHNOLOGY,53,3.970292
7,Jiangsu IoT Research and Development Center,48,3.871201
8,China Mobile,38,3.637586
9,Samsung Electronics,37,3.610918


In [16]:
com_ls = list(com_df.company)

In [17]:
com_ls.remove('Samsung Electronics')

# Data 2016

富比士2000資料

In [18]:
f_data = pd.read_csv("../../Forbes/Forbes_2016.csv")

In [19]:
f_data[f_data["Industry"]=="Semiconductors"]

,Company,Market Value,Revenue,Profits,Assets,Rank,Sector,Industry,Continent,Country,Headquarters,State,CEO,Forbes Webpage,Profits as % of Assets,Profits as % of Revenue
18,Samsung Electronics,161.550,177.286,16.5176,206.541,18,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Oh-Hyun Kwon,http://www.forbes.com/companies/samsung-electr...,0.079972,0.093169
59,Intel,149.259,56.276,11.4740,105.467,60,Information Technology,Semiconductors,North America,United States,NaN,NaN,Brian Krzanich,http://www.forbes.com/companies/intel/,0.108792,0.203888
136,Taiwan Semiconductor,125.605,25.661,9.0939,50.462,137,Information Technology,Semiconductors,Asia,Taiwan,Taiwan,NaN,Te Liu,http://www.forbes.com/companies/taiwan-semicon...,0.180213,0.354386
168,Qualcomm,77.324,22.614,4.9070,50.060,169,Information Technology,Semiconductors,North America,United States,California,California,Steven Mollenkopf,http://www.forbes.com/companies/qualcomm/,0.098022,0.216989
377,SK Hynix,17.513,16.609,3.8190,25.311,377,Information Technology,Semiconductors,Asia,South Korea,South Korea,NaN,Seong-Wook Park,http://www.forbes.com/companies/sk-hynix/,0.150883,0.229936
393,Texas Instruments,58.989,13.000,2.9440,16.230,394,Information Technology,Semiconductors,North America,United States,Texas,Texas,Richard Templeton,http://www.forbes.com/companies/texas-instrume...,0.181392,0.226462
559,NXP Semiconductors,29.199,6.101,1.5260,26.354,560,Information Technology,Semiconductors,Europe,Netherlands,Netherlands,NaN,Richard Clemmer,http://www.forbes.com/companies/nxp-semiconduc...,0.057904,0.250123
578,ASML Holding,41.325,6.972,1.7959,15.767,579,Information Technology,Semiconductors,Europe,Netherlands,Netherlands,NaN,Peter Wennink,http://www.forbes.com/companies/asml-holding/,0.113902,0.257587
582,Micron Technology,11.055,13.737,1.0710,24.819,583,Information Technology,Semiconductors,North America,United States,Idaho,Idaho,D Durcan,http://www.forbes.com/companies/micron-technol...,0.043152,0.077965
633,Applied Materials,23.858,9.557,1.3150,13.326,634,Information Technology,Semiconductors,North America,United States,California,California,Gary Dickerson,http://www.forbes.com/companies/applied-materi...,0.098679,0.137595


## Forbes list  
- 同Industry：1
- 同Sector：2
- 有出現在company list：3
- Company list上有但Forbes沒有：4

In [20]:
forb_ls =[]

for i in com_ls:
    if (f_data["Company"]==i).any():
        if (f_data[f_data.Company==i]["Industry"]=="Semiconductors").any():
            forb_ls.append(1)
        elif (f_data[f_data.Company==i]["Sector"]=="Information Technology").any():
            forb_ls.append(2)
        else:
            forb_ls.append(3)
    else:
        forb_ls.append(4)
    

In [21]:
forb_ls

[2, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4]

## Company list competitor

In [22]:
competitor_ls=[4, 2, 3, 3, 4, 3, 3, 3, 2, 1, 1, 1, 1, 1]

## 計算NMI

In [23]:
# (labels_true, labels_pred)
nmi = normalized_mutual_info_score(forb_ls,competitor_ls)
ari = adjusted_rand_score(forb_ls,competitor_ls)

print("nmi: ",nmi)
print("ari: ", ari)

nmi:  0.3455550601991344
ari:  0.14420062695924765


## 公司名稱和類別

In [24]:
competitor_df = pd.DataFrame((zip(com_ls, competitor_ls, forb_ls)), columns = ['company', '2016_competitor', '2016_forb'])

In [25]:
competitor_df

,company,2016_competitor,2016_forb
0,ZTE,4,2
1,Huawei Technologies,2,4
2,Anhui Sharetronic Iot Technology,3,4
3,Xian Irain IoT Technology Service,3,4
4,Shanghai Langbo Communication Technology,4,4
5,Nanjing IoT Sensor Technology,3,4
6,SUZHOU XUANHE IOT TECHNOLOGY,3,4
7,Jiangsu IoT Research and Development Center,3,4
8,China Mobile,2,3
9,Shenzhen JBT Smart Lighting,1,4


In [26]:
competitor_df.to_csv("../competitor_df/cpc_cn_2016.csv", index=False)